In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("./events.csv")

In [3]:
data["event_time"] = pd.to_datetime(data["event_time"].str[:19])
data

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06,view,1996170,2.140000e+18,electronics.telephone,NaN,31.90,1.520000e+18,LJuJVLEjPT
1,2020-09-24 11:57:26,view,139905,2.140000e+18,computers.components.cooler,zalman,17.16,1.520000e+18,tdicluNnRY
2,2020-09-24 11:57:27,view,215454,2.140000e+18,NaN,NaN,9.81,1.520000e+18,4TMArHtXQy
3,2020-09-24 11:57:33,view,635807,2.140000e+18,computers.peripherals.printer,pantum,113.81,1.520000e+18,aGFYrNgC08
4,2020-09-24 11:57:36,view,3658723,2.140000e+18,NaN,cameronsino,15.87,1.520000e+18,aa4mmk0kwQ
...,...,...,...,...,...,...,...,...,...
481994,2020-12-21 22:13:24,view,4154736,2.140000e+18,computers.peripherals.keyboard,roccat,113.17,1.520000e+18,aIgZll0RBP
481995,2020-12-21 22:14:15,view,4156223,2.140000e+18,electronics.audio.headphone,borofone,15.56,1.520000e+18,2nhXNJGCeS
481996,2020-12-21 22:15:59,view,4102122,2.140000e+18,NaN,baseus,13.65,1.520000e+18,KCb5plBKl6
481997,2020-12-21 22:18:03,view,1413341,2.140000e+18,computers.components.cpu,amd,191.52,1.520000e+18,rLnNMr2xIw


In [10]:
data_copy_1 = data

group_data = data_copy_1.groupby(["user_session", "event_type"]).agg(datetime = ("event_time", "min"))

temp_df = pd.DataFrame(data_copy_1['event_type'].unique(), columns = ["event_type"])
temp_df["step"] = [1, 2, 3]
temp_df.set_index("event_type", append = False, inplace = True)

merge_data = pd.merge(group_data, temp_df, left_on = "event_type", right_index = True)

pivot_data = merge_data.reset_index().pivot(index = ["user_session"], columns = "step", values = "datetime")

pivot_data.columns = temp_df.index
pivot_data

event_type,view,cart,purchase
user_session,,,
000c34fa-991f-442a-8e07-8c472269bec6,2020-09-30 17:52:02,NaT,NaT
001P7lK0Pt,2020-11-06 01:25:30,NaT,NaT
002DmERG1w,2020-09-26 18:59:06,NaT,NaT
003QqC0jk0,2020-10-25 15:23:25,NaT,NaT
003pEktS1X,2020-11-02 18:11:24,NaT,NaT
...,...,...,...
zztce59nU2,2020-10-29 18:02:28,NaT,NaT
zztjHPLepJ,2020-11-14 18:31:44,NaT,NaT
zzvjiUwHhM,2020-12-18 10:31:55,NaT,NaT


In [11]:
step_count = [pivot_data[event].notnull().sum() for event in pivot_data.columns]

print("event별 횟수")
for num, event in enumerate(step_count):
    print(pivot_data.columns[num], " : ", event, "회")
    
print("")

print("event별 전환율")
for num, event in enumerate(step_count[1:]):
    step_rate = round(event / step_count[num] * 100, 2)
    print(pivot_data.columns[num+1], " 전환율 : ", step_rate, "%")

event별 횟수
view  :  277475 회
cart  :  20870 회
purchase  :  12534 회

event별 전환율
cart  전환율 :  7.52 %
purchase  전환율 :  60.06 %


## 브랜드별 전환율

In [77]:
data.isna().sum()

event_time            0
event_type            0
product_id            0
category_id           0
category_code    138625
brand            119869
price                 0
user_id               0
user_session        132
dtype: int64

In [70]:
cart_data = data[data['event_type'] == 'cart']

purchase_data = data[data['event_type'] == 'purchase']

view_data = data[data['event_type'] == 'view']

cart_view_ratio = cart_data.groupby('brand').size() / view_data.groupby('brand').size()

purchase_cart_ratio = purchase_data.groupby('brand').size() / cart_data.groupby('brand').size()

purchase_view_ratio = purchase_data.groupby('brand').size() / view_data.groupby('brand').size()

result = pd.DataFrame({'cart_view_ratio': cart_view_ratio, 
                       'purchase_cart_ratio': purchase_cart_ratio, 
                       "purchase_view_ratio" : purchase_view_ratio,
                       "view_count" : view_data.groupby('brand').size(),
                       "cart_count" : cart_data.groupby('brand').size(),
                       "purchase_count" : purchase_data.groupby('brand').size()
                      }).reset_index()

print(result)

          brand  cart_view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0        a-data         0.030303             1.000000             0.030303   
1    accesstyle         0.500000             2.000000             1.000000   
2        accord         0.104278             0.666667             0.069519   
3          acct              NaN                  NaN                  NaN   
4       acecamp         0.111111             0.666667             0.074074   
..          ...              ...                  ...                  ...   
960         zmi              NaN                  NaN                  NaN   
961    zoweetek              NaN                  NaN                  NaN   
962       zowie              NaN                  NaN                  NaN   
963         zte         0.043182             0.421053             0.018182   
964       zyxel         0.059155             0.809524             0.047887   

     view_count  cart_count  purchase_count  
0            33  

In [70]:
result.isna().sum()

brand                    0
cart_view_ratio        475
purchase_cart_ratio    571
purchase_view_ratio    568
view_count               0
cart_count             475
purchase_count         568
dtype: int64

In [76]:
print("purchase / cart 전환율이 1보다 큰 경우의 수 :", len(result[result["purchase_cart_ratio"] > 1]))
result[result["purchase_cart_ratio"] > 1]

purchase / cart 전환율이 1보다 큰 경우의 수 : 45


,brand,cart_view_ratio,purchase_cart_ratio,purchase_view_ratio,view_count,cart_count,purchase_count
1,accesstyle,0.500000,2.000000,1.000000,2,1.0,2.0
7,acmepower,0.053333,1.750000,0.093333,75,4.0,7.0
14,adata,0.030612,1.333333,0.040816,392,12.0,16.0
132,candy,0.028986,1.500000,0.043478,207,6.0,9.0
135,canyon,0.108268,1.018182,0.110236,508,55.0,56.0
164,continent,0.037313,1.200000,0.044776,134,5.0,6.0
181,cyberflux,0.042373,1.200000,0.050847,118,5.0,6.0
188,darina,0.043478,1.500000,0.065217,46,2.0,3.0
219,dyp,0.071429,1.666667,0.119048,42,3.0,5.0
221,e.o.s,0.050000,1.500000,0.075000,40,2.0,3.0


In [71]:
result[result['cart_view_ratio'] == max(result['cart_view_ratio'])]

,brand,cart_view_ratio,purchase_cart_ratio,purchase_view_ratio,view_count,cart_count,purchase_count
1,accesstyle,0.5,2.0,1.0,2,1.0,2.0


In [72]:
result["brand"].unique()

array(['a-data', 'accesstyle', 'accord', 'acct', 'acecamp', 'acer',
       'achi', 'acmepower', 'acorp', 'acqua', 'activecam', 'activision',
       'acv', 'adaptec', 'adata', 'aerocool', 'agestar', 'aimoto',
       'airmax', 'airwheel', 'ajax', 'akel', 'akenori', 'alan', 'alas',
       'alcatel', 'alfa', 'alienware', 'alligator', 'allocacoc',
       'alpenkok', 'alphachem', 'alpine', 'amazfit', 'amd', 'amega',
       'and', 'andis', 'anker', 'anl', 'anymode', 'anytek', 'aoc',
       'aopen', 'aorus', 'aosenma', 'apacer', 'apc', 'apexto', 'apple',
       'aqc', 'araree', 'arctic', 'arian', 'ark', 'artway', 'asahi',
       'asrock', 'astralux', 'asus', 'asustor', 'asx', 'aten', 'atis',
       'atrix', 'auchan', 'audio-technica', 'aukey', 'aura', 'autoprofi',
       'avaya', 'avengers', 'avermedia', 'avinity', 'avis', 'avision',
       'axon', 'b.well', 'bakon', 'baku', 'ballistix', 'ballu', 'baofeng',
       'barva', 'baseus', 'batterika', 'battleground', 'bayerhoff', 'bb',
       'bbk',

In [76]:
result[result["brand"] == "htc"]
result[result["brand"] == "hp"]
result[result["brand"] == "samsung"]

,brand,cart_view_ratio,purchase_cart_ratio,purchase_view_ratio,view_count,cart_count,purchase_count
739,samsung,0.044937,0.73945,0.033229,12128,545.0,403.0


### 분석

의외로 브랜드별 집계에서 cart의 size 보다 purchase의 size가 더 크다.
이는 특정 브랜드를 검색해서 해당 상품을 바로 구매하는 경우이거나
혹은 브랜드와 무관하게 특정 상품만을 검색하고 구매한 경우가 아닌가 생각됨.
그런데 cart / purchase가 1보다 큰 경우에 해당하는 브랜드들이 네임밸류가 약한 브랜드들이라 의문

## 카테고리별 전환율

In [19]:
data_copy_category = data.copy()
data_copy_category.dropna(axis=0, how="any", inplace=True)

In [24]:
split_values = data_copy_category["category_code"].str.split('.', expand=True)
split_values

,0,1,2,3
1,computers,components,cooler,None
3,computers,peripherals,printer,None
5,construction,tools,saw,None
7,computers,network,router,None
9,computers,network,router,None
...,...,...,...,...
481993,construction,tools,welding,None
481994,computers,peripherals,keyboard,None
481995,electronics,audio,headphone,None
481997,computers,components,cpu,None


In [34]:
data_category = pd.concat([data_copy_category, split_values], axis=1)
data_category = data_category.rename(columns={0:"category_LV1", 1:"category_LV2", 2: "category_LV3", 3:"category_LV4"})
data_category

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_LV1,category_LV2,category_LV3,category_LV4
1,2020-09-24 11:57:26,view,139905,2.140000e+18,computers.components.cooler,zalman,17.16,1.520000e+18,tdicluNnRY,computers,components,cooler,None
3,2020-09-24 11:57:33,view,635807,2.140000e+18,computers.peripherals.printer,pantum,113.81,1.520000e+18,aGFYrNgC08,computers,peripherals,printer,None
5,2020-09-24 11:57:59,view,664325,2.140000e+18,construction.tools.saw,carver,52.33,1.520000e+18,vnkdP81DDW,construction,tools,saw,None
7,2020-09-24 11:58:24,view,716611,2.140000e+18,computers.network.router,d-link,53.14,1.520000e+18,kVBeYDPcBw,computers,network,router,None
9,2020-09-24 11:58:31,view,716611,2.140000e+18,computers.network.router,d-link,53.14,1.520000e+18,F3VB9LYp39,computers,network,router,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
481993,2020-12-21 22:12:31,view,1000116,2.140000e+18,construction.tools.welding,fubag,228.52,1.520000e+18,H7MkpEtWaq,construction,tools,welding,None
481994,2020-12-21 22:13:24,view,4154736,2.140000e+18,computers.peripherals.keyboard,roccat,113.17,1.520000e+18,aIgZll0RBP,computers,peripherals,keyboard,None
481995,2020-12-21 22:14:15,view,4156223,2.140000e+18,electronics.audio.headphone,borofone,15.56,1.520000e+18,2nhXNJGCeS,electronics,audio,headphone,None
481997,2020-12-21 22:18:03,view,1413341,2.140000e+18,computers.components.cpu,amd,191.52,1.520000e+18,rLnNMr2xIw,computers,components,cpu,None


In [41]:
cart_data = data_category[data_category['event_type'] == 'cart']

purchase_data = data_category[data_category['event_type'] == 'purchase']

view_data = data_category[data_category['event_type'] == 'view']

cart_view_ratio = cart_data.groupby('category_LV1').size() / view_data.groupby('category_LV1').size()

purchase_cart_ratio = purchase_data.groupby('category_LV1').size() / cart_data.groupby('category_LV1').size()

purchase_view_ratio = purchase_data.groupby('category_LV1').size() / view_data.groupby('category_LV1').size()

result = pd.DataFrame({'cart_view_ratio': cart_view_ratio, 
                       'purchase_cart_ratio': purchase_cart_ratio, 
                       "purchase_view_ratio" : purchase_view_ratio,
                       "view_count" : view_data.groupby('category_LV1').size(),
                       "cart_count" : cart_data.groupby('category_LV1').size(),
                       "purchase_count" : purchase_data.groupby('category_LV1').size()
                      }).reset_index()

print(result)

    category_LV1  cart_view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0    accessories         0.033778             0.789474             0.026667   
1        apparel         0.027027                  NaN                  NaN   
2     appliances         0.031522             0.776398             0.024474   
3           auto         0.044444             0.733696             0.032609   
4      computers         0.090679             0.657636             0.059634   
5   construction         0.046618             0.720559             0.033591   
6   country_yard         0.016445             0.538462             0.008855   
7    electronics         0.045729             0.715405             0.032715   
8      furniture         0.011236                  NaN                  NaN   
9        jewelry              NaN                  NaN                  NaN   
10          kids         0.038961             0.888889             0.034632   
11      medicine         0.033149             0.6666

In [51]:
result[result['cart_view_ratio'] == max(result['cart_view_ratio'])]
result[result['cart_view_ratio'] == min(result['cart_view_ratio'])]
result[result['purchase_cart_ratio'] == max(result['purchase_cart_ratio'])]
result[result['purchase_cart_ratio'] == min(result['purchase_cart_ratio'])]
result[result['purchase_view_ratio'] == max(result['purchase_view_ratio'])]
result[result['purchase_view_ratio'] == min(result['purchase_view_ratio'])]

,category_LV1,cart_view_ratio,purchase_cart_ratio,purchase_view_ratio,view_count,cart_count,purchase_count
6,country_yard,0.016445,0.538462,0.008855,1581,26.0,14.0


In [54]:
cart_data = data_category[data_category['event_type'] == 'cart']

purchase_data = data_category[data_category['event_type'] == 'purchase']

view_data = data_category[data_category['event_type'] == 'view']

cart_view_ratio = cart_data.groupby('category_LV2').size() / view_data.groupby('category_LV2').size()

purchase_cart_ratio = purchase_data.groupby('category_LV2').size() / cart_data.groupby('category_LV2').size()

purchase_view_ratio = purchase_data.groupby('category_LV2').size() / view_data.groupby('category_LV2').size()

result = pd.DataFrame({'cart_view_ratio': cart_view_ratio, 
                       'purchase_cart_ratio': purchase_cart_ratio, 
                       "purchase_view_ratio" : purchase_view_ratio,
                       "view_count" : view_data.groupby('category_LV2').size(),
                       "cart_count" : cart_data.groupby('category_LV2').size(),
                       "purchase_count" : purchase_data.groupby('category_LV2').size()
                      }).reset_index()

print(result)

       category_LV2  cart_view_ratio  purchase_cart_ratio  \
0       accessories         0.044444             0.733696   
1             audio         0.044895             0.811071   
2               bag         0.033778             0.789474   
3          bathroom         0.050000                  NaN   
4           battery         0.041199             0.863636   
5           bedroom              NaN                  NaN   
6           bicycle         0.074830             0.363636   
7        calculator         0.049383             1.250000   
8            camera         0.062280             0.773585   
9          cartrige         0.093050             0.891425   
10           clocks         0.018747             0.542857   
11       components         0.099842             0.566556   
12       cultivator         0.012955             0.312500   
13          desktop         0.063934             0.658120   
14           ebooks         0.098266             0.800000   
15      environment     

In [56]:
cart_data = data_category[data_category['event_type'] == 'cart']

purchase_data = data_category[data_category['event_type'] == 'purchase']

view_data = data_category[data_category['event_type'] == 'view']

cart_view_ratio = cart_data.groupby('category_LV3').size() / view_data.groupby('category_LV3').size()

purchase_cart_ratio = purchase_data.groupby('category_LV3').size() / cart_data.groupby('category_LV3').size()

purchase_view_ratio = purchase_data.groupby('category_LV3').size() / view_data.groupby('category_LV3').size()

result = pd.DataFrame({'cart_view_ratio': cart_view_ratio, 
                       'purchase_cart_ratio': purchase_cart_ratio, 
                       "purchase_view_ratio" : purchase_view_ratio,
                       "view_count" : view_data.groupby('category_LV3').size(),
                       "cart_count" : cart_data.groupby('category_LV3').size(),
                       "purchase_count" : purchase_data.groupby('category_LV3').size()
                      }).reset_index()

print(result)

   category_LV3  cart_view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0      acoustic         0.046084             0.825944             0.038063   
1    air_heater         0.018100             1.000000             0.018100   
2         alarm         0.013997             0.222222             0.003110   
3          bath         0.050000                  NaN                  NaN   
4       blanket              NaN                  NaN                  NaN   
..          ...              ...                  ...                  ...   
72      welding         0.048401             0.642857             0.031115   
73         wifi         0.084549             0.856230             0.072393   
74        winch              NaN                  NaN                  NaN   
75       window              NaN                  NaN                  NaN   
76       wrench         0.039801             1.000000             0.039801   

    view_count  cart_count  purchase_count  
0        13215    

### 분석

카테고리1,2에 집중해서 살펴보면, 컴퓨터 부분은 cart에는 넣는 경우가 많으나 실제 구매 확률은 상대적으로 적은 반면, kids 의복 문구의 경우 view -> cart에 비해 실제 구매전환율이 상대적으로 더 높다. 이는 홈페이지에 등록된 컴퓨터 부품보다 kids, 의복 문구의 아이템이 더 많거나 혹은 컴퓨터 부품의 표본이 더 커서 이러한 오차가 발생했을 가능성이 있다. 다만, 문구나 kids를 보면 cart에 담긴 수보다 purchase가 더 높은 항목들이 있는데 이는 부모가 아이를 대신해서 특정 제품을 바록 구매하거나 특정 제품이 압도적인 영향력을 행사하는 경우가 아닐까 생각된다.

## 시간별 전환율

In [63]:
# event_time 컬럼 str -> datetime64로 변경
data_copy = data.copy()
data_copy["event_time"] = pd.to_datetime(data_copy["event_time"])

# event_time 컬럼 월, 일, 요일, 시로 분리
data_copy["month"] = data_copy["event_time"].dt.month
data_copy["day"] = data_copy["event_time"].dt.day
data_copy["day_name"] = data_copy["event_time"].apply(lambda x : x.day_name())
data_copy["hour"] = data_copy["event_time"].dt.hour

col_order = ['event_time', 'month', 'day', 'day_name', 'hour', 
             'event_type', 'product_id', 'category_id', 'category_code', 
             'brand', 'price', 'user_id', 'user_session']
data_copy = data_copy[col_order]
data_copy.head()

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06,9,24,Thursday,11,view,1996170,2.140000e+18,electronics.telephone,NaN,31.90,1.520000e+18,LJuJVLEjPT
1,2020-09-24 11:57:26,9,24,Thursday,11,view,139905,2.140000e+18,computers.components.cooler,zalman,17.16,1.520000e+18,tdicluNnRY
2,2020-09-24 11:57:27,9,24,Thursday,11,view,215454,2.140000e+18,NaN,NaN,9.81,1.520000e+18,4TMArHtXQy
3,2020-09-24 11:57:33,9,24,Thursday,11,view,635807,2.140000e+18,computers.peripherals.printer,pantum,113.81,1.520000e+18,aGFYrNgC08
4,2020-09-24 11:57:36,9,24,Thursday,11,view,3658723,2.140000e+18,NaN,cameronsino,15.87,1.520000e+18,aa4mmk0kwQ


In [64]:
## month

cart_data = data_copy[data_copy['event_type'] == 'cart']

purchase_data = data_copy[data_copy['event_type'] == 'purchase']

view_data = data_copy[data_copy['event_type'] == 'view']

cart_view_ratio = cart_data.groupby('month').size() / view_data.groupby('month').size()

purchase_cart_ratio = purchase_data.groupby('month').size() / cart_data.groupby('month').size()

purchase_view_ratio = purchase_data.groupby('month').size() / view_data.groupby('month').size()

result = pd.DataFrame({'cart_view_ratio': cart_view_ratio, 
                       'purchase_cart_ratio': purchase_cart_ratio, 
                       "purchase_view_ratio" : purchase_view_ratio,
                       "view_count" : view_data.groupby('month').size(),
                       "cart_count" : cart_data.groupby('month').size(),
                       "purchase_count" : purchase_data.groupby('month').size()
                      }).reset_index()

print(result)

   month  cart_view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0      9         0.055038             0.713173             0.039252   
1     10         0.059568             0.718983             0.042828   
2     11         0.061304             0.725827             0.044496   
3     12         0.066674             0.703977             0.046937   

   view_count  cart_count  purchase_count  
0       25655        1412            1007  
1      146539        8729            6276  
2      170216       10435            7574  
3       93530        6236            4390  


In [69]:
## 요일

to_cart__from_view_ratio = cart_data.groupby('day_name').size() / view_data.groupby('day_name').size()

to_purchase_from_cart_ratio = purchase_data.groupby('day_name').size() / cart_data.groupby('day_name').size()

to_purchase_from_view_ratio = purchase_data.groupby('day_name').size() / view_data.groupby('day_name').size()

result = pd.DataFrame({'cart_view_ratio': to_cart__from_view_ratio, 
                       'purchase_cart_ratio': to_purchase_from_cart_ratio, 
                       "purchase_view_ratio" : to_purchase_from_view_ratio,
                       "view_count" : view_data.groupby('day_name').size(),
                       "cart_count" : cart_data.groupby('day_name').size(),
                       "purchase_count" : purchase_data.groupby('day_name').size()
                      }).reset_index()

print(result)

    day_name  cart_view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0     Friday         0.061197             0.715646             0.043795   
1     Monday         0.062217             0.724831             0.045097   
2   Saturday         0.059664             0.719035             0.042901   
3     Sunday         0.059287             0.708157             0.041985   
4   Thursday         0.062021             0.714650             0.044323   
5    Tuesday         0.062198             0.709402             0.044123   
6  Wednesday         0.063644             0.731651             0.046566   

   view_count  cart_count  purchase_count  
0       63500        3886            2781  
1       68808        4281            3103  
2       56969        3399            2444  
3       59760        3543            2509  
4       63172        3918            2800  
5       62076        3861            2739  
6       61655        3924            2871  


In [67]:
## 시간대

to_cart__from_view_ratio = cart_data.groupby('hour').size() / view_data.groupby('hour').size()

to_purchase_from_cart_ratio = purchase_data.groupby('hour').size() / cart_data.groupby('hour').size()

to_purchase_from_view_ratio = purchase_data.groupby('hour').size() / view_data.groupby('hour').size()

result = pd.DataFrame({'cart__view_ratio': to_cart__from_view_ratio, 
                       'purchase_cart_ratio': to_purchase_from_cart_ratio, 
                       "purchase_view_ratio" : to_purchase_from_view_ratio,
                       "view_count" : view_data.groupby('hour').size(),
                       "cart_count" : cart_data.groupby('hour').size(),
                       "purchase_count" : purchase_data.groupby('hour').size()
                      }).reset_index()

print(result)

    hour  cart__view_ratio  purchase_cart_ratio  purchase_view_ratio  \
0      0          0.056921             0.505837             0.028793   
1      1          0.050575             0.643777             0.032559   
2      2          0.054210             0.690722             0.037444   
3      3          0.059345             0.683258             0.040548   
4      4          0.059627             0.746106             0.044488   
5      5          0.060379             0.673203             0.040647   
6      6          0.061664             0.732327             0.045158   
7      7          0.064425             0.710492             0.045773   
8      8          0.064994             0.743401             0.048316   
9      9          0.065297             0.724016             0.047276   
10    10          0.066616             0.805825             0.053681   
11    11          0.066779             0.735547             0.049119   
12    12          0.062853             0.743574             0.04

### 분석

월별로는 12월, 요일로는 수요일에 가장 퍼포먼스가 좋고 시간대별로는 10시 부근이 가장 좊으며 전반적으로 새벽보다 밤에 더 purchase 전환율이 높은 경향을 보인다.

#제언

<결과>
1. 브랜드별 분석 결과, 특정 브랜드가 사용자의 구매 절차에 영향을 줄 가능성이 있으나 브랜드보다 특정 품목에 종속되어 있을 가능성이 있다.
2. 위 결과를 고려하여 카테고리별로 분석한 결과 가장 많은 표본을 갖고 있는 computer는 cart 비율이 높으나 purchase 비율이 낮고 그 반대에 해당하는 카테고리들이 존재한다.
3. 시간대별로는 오전 10시, 밤 시간대의 cart 및 purchase 전환율이 높은데 반해 퇴근 시간은 둘 다 적고 새벽은 cart에 비해 purchase가 적다.

<제언>
1. 특정 브랜드 별 구매전환율의 유의미한 차이는 보이지 않는다. 따라서 브랜드에 집중하는 것은 좋은 전략이 아닌 것으로 보인다.
2. computer 부품의 경우 cart에 담는 비율을 높지만 실제 purchase의 비율은 낮기 때문에 이를 개선하기 위해 일정 기간마다 cart에 담긴 아이템 현황 알람을 보내는 등의 조치를 생각해볼 수 있을 것 같다. 또한 purchase 비율이 높은 카테고리들 (kids, 의류, 문구)의 조회수를 높이기 위해 이벤트 등을 생각해볼 수 있을 것
3. 분석 결과 구매 전환율을 높이기 위해서는 view는 높지만 cart에 담거나 purchase에 담는 비율이 낮은 새벽 시간대에 구매를 촉진시킬 필요